In [1]:
! pip install langchain_core langchain_community langchain_groq load_dotenv pinecone

  Using cached langchain_core-0.3.47-py3-none-any.whl.metadata (5.9 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.5.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (13 kB)
  Using cached yarl-1.18.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (69 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached orjson-3.10.15-cp312-cp312-macosx_10_15_x86_64.macosx_11_0_arm64.macosx_10_15_universal2.whl.metadata (41 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached zst

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_groq import ChatGroq
from load_dotenv import load_dotenv
from pinecone import Pinecone ,ServerlessSpec
import re 
import os
load_dotenv()

/opt/homebrew/Caskroom/miniconda/base/envs/analysis_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [ ]:
def clean_text(text: str) -> str:
    """Clean the text from the input file.
    Args:
        text (str): The text to clean.
    Returns:
        str: The cleaned text.
    """
    # Remove carriage returns
    text = text.replace("\r", "")

    # Remove extra spaces
    text = re.sub(" +", " ", text)

    # Replace multiple newlines with a single newline
    text = re.sub("\n+", "\n", text)

    # Remove leading and trailing whitespace
    text = text.strip()

    # Remove unwanted characters (non-ASCII characters)
    text = "".join(i for i in text if ord(i) < 128)

    # remove multiple punctuations with a single punctuation
    text = re.sub(r'(\W)\1+', r'\1', text)

    return text



In [3]:
def get_text_from_file(file_path: str) -> str:
    """Read the text from the input file.
    Args:
        file_path (str): The path to the input file.
    Returns:
        str: The text read from the input file.
    """
    with open(file_path, "r") as file:
        text = file.read()
    return text

In [5]:
# read hotel_booking.txt file
file_path = "hotel_bookings_detailed_report.txt"
text = get_text_from_file(file_path)
text

'Hotel Bookings Detailed Report\n\nData covers from 2015-07-01 to 2017-08-31\n\nTotal Bookings- 119390\nNon-Canceled Bookings- 75166\n\nMonthly revenues-\n2015-07- 758339.79\n2015-08- 1137652.71\n2015-09- 1054620.67\n2015-10- 784714.88\n2015-11- 346709.49\n2015-12- 429521.57\n2016-01- 264521.38\n2016-02- 484170.72\n2016-03- 767337.42\n2016-04- 896591.38\n2016-05- 1073277.63\n2016-06- 1144800.38\n2016-07- 1525019.05\n2016-08- 1809324.75\n2016-09- 1289642.69\n2016-10- 1072101.84\n2016-11- 688843.47\n2016-12- 657870.72\n2017-01- 512155.46\n2017-02- 663370.76\n2017-03- 884761.17\n2017-04- 1203266.10\n2017-05- 1321936.01\n2017-06- 1438490.54\n2017-07- 1817038.23\n2017-08- 1970181.60\n\nOverall Cancellation Rate- 37.04%\n\nTop 10 Countries by Booking Percentage-\nPRT- 40.70%\nGBR- 10.16%\nFRA- 8.72%\nESP- 7.18%\nDEU- 6.10%\nITA- 3.15%\nIRL- 2.83%\nBEL- 1.96%\nBRA- 1.86%\nNLD- 1.76%\n\nLead Time Distribution-\ncount- 119390.0\nmean- 104.01141636652986\nstd- 106.86309704798794\nmin- 0.0\n25%- 

In [8]:
clean_text(text)

'Hotel Bookings Detailed Report\nData covers from 2015-07-01 to 2017-08-31\nTotal Bookings- 119390\nNon-Canceled Bookings- 75166\nMonthly revenues-\n2015-07- 758339.79\n2015-08- 1137652.71\n2015-09- 1054620.67\n2015-10- 784714.88\n2015-11- 346709.49\n2015-12- 429521.57\n2016-01- 264521.38\n2016-02- 484170.72\n2016-03- 767337.42\n2016-04- 896591.38\n2016-05- 1073277.63\n2016-06- 1144800.38\n2016-07- 1525019.05\n2016-08- 1809324.75\n2016-09- 1289642.69\n2016-10- 1072101.84\n2016-11- 688843.47\n2016-12- 657870.72\n2017-01- 512155.46\n2017-02- 663370.76\n2017-03- 884761.17\n2017-04- 1203266.10\n2017-05- 1321936.01\n2017-06- 1438490.54\n2017-07- 1817038.23\n2017-08- 1970181.60\nOverall Cancellation Rate- 37.04%\nTop 10 Countries by Booking Percentage-\nPRT- 40.70%\nGBR- 10.16%\nFRA- 8.72%\nESP- 7.18%\nDEU- 6.10%\nITA- 3.15%\nIRL- 2.83%\nBEL- 1.96%\nBRA- 1.86%\nNLD- 1.76%\nLead Time Distribution-\ncount- 119390.0\nmean- 104.01141636652986\nstd- 106.86309704798794\nmin- 0.0\n25%- 18.0\n50%- 6

In [ ]:
# Get info about the text file